In [1]:
import pickle
import pandas as pd
import numpy as np
import os, sys, gc 
from plotnine import *
import plotnine

from tqdm import tqdm_notebook
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib as mpl
from matplotlib import rc
import re
from matplotlib.ticker import PercentFormatter
import datetime
from math import log # IDF 계산을 위해

In [2]:
path = 'C:/Users/User/Documents/T아카데미/T 아카데미/input/'

In [3]:
# pd.read_json : json 형태의 파일을 dataframe 형태로 불러오는 코드 
magazine = pd.read_json(path + 'magazine.json', lines=True) # lines = True : Read the file as a json object per line.
metadata = pd.read_json(path + 'metadata.json', lines=True)
users = pd.read_json(path + 'users.json', lines=True)

In [4]:
%%time 
import itertools
from itertools import chain
import glob
import os 

input_read_path = path + 'read/read/'
# os.listdir : 해당 경로에 있는 모든 파일들을 불러오는 명령어 
file_list = os.listdir(input_read_path)
exclude_file_lst = ['read.tar', '.2019010120_2019010121.un~']

read_df_list = []
for file in tqdm_notebook(file_list):
    # 예외처리 
    if file in exclude_file_lst:
        continue 
    else:
        file_path = input_read_path + file
        df_temp = pd.read_csv(file_path, header=None, names=['raw'])
        # file명을 통해서 읽은 시간을 추출(from, to)
        df_temp['from'] = file.split('_')[0]
        df_temp['to'] = file.split('_')[1]
        read_df_list.append(df_temp)
    
read_df = pd.concat(read_df_list)
# reads 파일을 전처리해서 row 당 user - article이 1:1이 되도록 수정 
read_df['user_id'] = read_df['raw'].apply(lambda x: x.split(' ')[0])
read_df['article_id'] = read_df['raw'].apply(lambda x: x.split(' ')[1:])

def chainer(s):
    return list(itertools.chain.from_iterable(s))

read_cnt_by_user = read_df['article_id'].map(len)
read_rowwise = pd.DataFrame({'from': np.repeat(read_df['from'], read_cnt_by_user),
                             'to': np.repeat(read_df['to'], read_cnt_by_user),
                             'user_id': np.repeat(read_df['user_id'], read_cnt_by_user),
                             'article_id': chainer(read_df['article_id'])})

read_rowwise.reset_index(drop=True, inplace=True)


Wall time: 25 s


In [5]:
from datetime import datetime 

metadata['reg_datetime'] = metadata['reg_ts'].apply(lambda x : datetime.fromtimestamp(x/1000.0))
metadata.loc[metadata['reg_datetime'] == metadata['reg_datetime'].min(), 'reg_datetime'] = datetime(2090, 12, 31)
metadata['reg_dt'] = metadata['reg_datetime'].dt.date
metadata['type'] = metadata['magazine_id'].apply(lambda x : '개인' if x == 0.0 else '매거진')
metadata['reg_dt'] = pd.to_datetime(metadata['reg_dt'])

In [6]:
read_rowwise = read_rowwise.merge(metadata[['id', 'reg_dt']], how='left', left_on='article_id', right_on='id')
read_rowwise = read_rowwise[read_rowwise['article_id'] != '']

# 사용자가 읽은 글의 목록들을 저장 
read_total = pd.DataFrame(read_rowwise.groupby(['user_id'])['article_id'].unique()).reset_index()
read_total.columns = ['user_id', 'article_list']

## 콘텐츠 기반의 추천시스템
- Model의 단어를 이용한 방식
- TF-IDF 형식
    - index : 문서의 아이디 
    - column : 단어 

하지만, 문서가 총 64만개로 너무 많고 data.0의 파일을 읽어보면 단어 또한 너무 많아서 사용하기가 어려운 상황

### 해결방식
위와 같은 문제를 해결하기 위해서 해당 대회의 1등팀인 NAFMA팀은 글의 키워드를 활용해서 Embedding을 구성 
- 참고자료 : https://github.com/JungoKim/brunch_nafma

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

metadata = metadata[metadata['keyword_list'].notnull()].reset_index()
metadata = metadata[metadata['reg_dt'] >= '2019-01-01']

In [8]:
article2idx = {}
for i, l in enumerate(metadata['id'].unique()):
    article2idx[l] = i
    
idx2article = {i: item for item, i in article2idx.items()}
articleidx = metadata['articleidx'] = metadata['id'].apply(lambda x: article2idx[x]).values

In [9]:
import scipy

docs = metadata['keyword_list'].apply(lambda x: ' '.join(x)).values
tfidv = TfidfVectorizer(use_idf=True, smooth_idf=False, norm=None).fit(docs)
tfidv_df = scipy.sparse.csr_matrix(tfidv.transform(docs))
tfidv_df = tfidv_df.astype(np.float32)

In [10]:
print(tfidv_df.shape)

(73574, 20823)


데이터가 Sparse 형태인 것을 확인할 수 있음

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

# 메모리 문제 발생 
cos_sim = cosine_similarity(tfidv_df, tfidv_df)

In [12]:
valid = pd.read_csv(path + '/predict/predict/dev.users', header=None)

In [13]:
%%time 
popular_rec_model = read_rowwise['article_id'].value_counts().index[0:100]

top_n = 100
with open('./recommend.txt', 'w') as f:
    for user in tqdm_notebook(valid[0].values):
        seen = chainer(read_total[read_total['user_id'] == user]['article_list'])
        for seen_id in seen:
            # 2019년도 이전에 읽어서 혹은 메타데이터에 글이 없어서 유사도 계산이 안된 글
            cos_sim_sum = np.zeros(len(cos_sim))
            try:
                cos_sim_sum += cos_sim[article2idx[seen_id]]
            except:
                pass

        recs = []
        for rec in cos_sim_sum.argsort()[-(top_n+100):][::-1]:
            if (idx2article[rec] not in seen) & (len(recs) < 100):
                recs.append(idx2article[rec])

        f.write('%s %s\n' % (user, ' '.join(recs[0:100])))


Wall time: 2min 41s


![](https://github.com/choco9966/T-academy-Recommendation/blob/master/figure/Contents_Based_Score.PNG?raw=true)